<a href="https://colab.research.google.com/github/AnamKwon/Dacon/blob/main/Text_ext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch==1.3.1
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |███████████████████████████████▌| 834.1MB 1.3MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x39cce000 @  0x7efe80c25615 0x591e47 0x4cc179 0x4cc2db 0x50a1cc 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd
     |████████████████████████████████| 846.0MB 21kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 9.9MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588510 sha256=bcdfb7447690d94befed1aff791aa80f49fa271d4976df8cdee681629231c4aa
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.0MB 8.7MB

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule
import json
import transformers
from google.colab import drive
drive.mount('/content/drive/')

device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
using cached model
using cached model
using cached model


##KOBERT Pre-trained 불러오기

In [ ]:
with open('/content/drive/MyDrive/dacon_ExtSum/train.jsonl') as f :
  train_data = [json.loads(i.strip()) for i in f.readlines()]

MAX_LEN, count_SENTENTS = (512, 70)
df = []
labels = []
for d in train_data :
  token_ids = [2]
  segment_ids = [0]
  attention_mask = [1]
  label = d['extractive']
  y = [0] * count_SENTENTS
  data = d['article_original']
  segment = 0
  for i in data :
    if len(token_ids) > 511 :
      token_ids = token_ids[:511]+[3]
      segment_ids = segment_ids[:511]+[segment]
      break
    token = tok(i)+['[SEP]']
    ids = tok.convert_tokens_to_ids(token)
    token_ids += ids
    if segment == 0 :
      segment_ids += [0]*len(ids)
      segment = 1
    else :
      segment_ids += [1]*len(ids)
      segment = 0
  for i in label :
    if int(i) >= count_SENTENTS : continue;
    y[int(i)] = 1
  token_ids = token_ids + [1] * (MAX_LEN - len(token_ids))
  attention_mask = [1]* len(segment_ids) + [0] * (MAX_LEN - len(segment_ids))
  segment_ids = segment_ids + [segment] * (MAX_LEN - len(segment_ids))
  df.append((torch.tensor(token_ids[:512]).long(),torch.tensor(attention_mask[:512]).long(),torch.tensor(segment_ids[:512]).long(),torch.tensor(y).float()))

## 데이터 셋 
# Segment_ids == 문장구분
# attention_mask == 문자열 구분(padding 된 값 계산에 활용하지 않게)
# token_ids == Bert 토크나이저로 벡터화된 문장
벡터화된 문장 사이에 [SEP] 태그를 넣어줌으로써 문장 구분


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        # self.dense1 = nn.Linear(768,384,bias=False)
        # self.dense2 = nn.Linear(384,192,bias=False)
        # self.dense3 = nn.Linear(192,96,bias=False)
        self.classifier = nn.Linear(768,70,bias=False)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def forward(self, token_ids, valid_length, segment_ids):
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids, attention_mask = valid_length)
        if self.dr_rate:
            out = self.dropout(pooler)
        # out = self.dense1(out)
        # out = self.dropout(out)
        # out = self.dense2(out)
        # out = self.dropout(out)
        # out = self.dense3(out)
        return self.classifier(out)


def binary_acc(y_pred, y_test):
    check = 0
    for x in range(len(y_pred)) :
      y = y_pred[x]
      idx = y.argmax()
      if y_test[x][idx] == 1:
        check += 1
      y[idx] = -10
      idx = y.argmax()
      if y_test[x][idx] == 1:
        check += 1
      y[idx] = -10
      idx = y.argmax()
      if y_test[x][idx] == 1:
        check += 1
    acc = check/30
    
    return acc

## Bert fine turing 모델 생성
처음에는 여러층을 쌓아서 진행하였지만, 한개의 dense레이어를 활용했을 경우 가장 높은 점수를 가져왔다.

문제점 : 전체 문장을 읽어오기 위해선 512개의 벡터로는 모든 문장을 가져올수 없지만, 앞에 문장을 읽어와 확인 하는 것으로 변경

활성함수를 넣지 않을 이유 : pytorch의 loss function(CrossEntropy(단일 라벨), BCEWithLogits(다중라벨))에서 활성화함수를 적용하여 계산

In [ ]:
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
train_set = torch.utils.data.DataLoader(df,batch_size=10)
model = BERTClassifier(bertmodel,dr_rate=0.5).to(device)
#model.load_state_dict(torch.load('/content/drive/MyDrive/dacon_ExtSum/checkpoint_ext_bias_false.tar',map_location=torch.device('cpu'))['model_state_dict'])
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.BCEWithLogitsLoss()
t_total = len(train_set)
warmup_step = int(t_total * warmup_ratio)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

##모델, Loss Function, Optimizer 선언

In [ ]:
for e in range(5):
    train_acc = 0.0
    test_acc = 0.0
    loss_val = []
    acc_val = []
    train_set = torch.utils.data.DataLoader(df,batch_size=10,shuffle=True)
    model.train()
    for batch_id, (token_ids, attention_mask,token_type, label) in enumerate(tqdm_notebook(train_set)):
        optimizer.zero_grad()
        token_ids = token_ids.to(device)
        token_type = token_type.to(device)
        attention_mask = attention_mask.to(device)
        label = label.float().to(device)
        out = model(token_ids, attention_mask, token_type)
        loss = loss_fn(out, label)
        loss.backward()
        loss_val.append(loss.item())
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        acc = binary_acc(out,label)
        acc_val.append(acc)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        #train_acc += binary_acc(out,label)
        #train_acc += calc_accuracy(out, label)
        if batch_id % 200 == 0:
            print("epoch {} batch id {} loss {} acc {}".format(e+1, batch_id+1,  sum(loss_val)/len(loss_val), sum(acc_val)/len(acc_val)))
        if batch_id == 2000 :
            torch.save({
                        'epoch': e,
                        'train_no': batch_id,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss':loss
                      }, '/content/drive/MyDrive/dacon_ExtSum/checkpoint_ext_70.tar')
    print("epoch {} loss {} acc {}".format(e+1, sum(loss_val)/len(loss_val), sum(acc_val)/len(acc_val)))
    torch.save({
            'epoch': e,
            'train_no': batch_id,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss':loss
          }, '/content/drive/MyDrive/dacon_ExtSum/checkpoint_ext_70.tar')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


epoch 1 batch id 1 loss 0.07208019495010376 acc 0.6333333333333333
epoch 1 batch id 201 loss 0.08070251113962178 acc 0.6024875621890549
epoch 1 batch id 401 loss 0.08107620352261381 acc 0.5998337489609288


KeyboardInterrupt: ignored

## Training 과정, 중간중간 체크포인트 저장

In [ ]:
with open('/content/drive/MyDrive/dacon_ExtSum/extractive_test_v2.jsonl') as f :
  test_data = [json.loads(i.strip()) for i in f.readlines()]
test = []
MAX_LEN = 512
for d in test_data :
  token_ids = [2]
  segment_ids = [0]
  attention_mask = [1]
  data = d['article_original']
  id = d['id']
  segment = 0
  for i in data :
    if len(token_ids) > 511 :
      token_ids = token_ids[:511]+[3]
      segment_ids = segment_ids[:511]+[segment]
      break
    token = tok(i)+['[SEP]']
    ids = tok.convert_tokens_to_ids(token)
    token_ids += ids
    if segment == 0 :
      segment_ids += [0]*len(ids)
      segment = 1
    else :
      segment_ids += [1]*len(ids)
      segment = 0
  token_ids = token_ids + [1] * (MAX_LEN - len(token_ids))
  attention_mask = [1]* len(segment_ids) + [0] * (MAX_LEN - len(segment_ids))
  segment_ids = segment_ids + [segment] * (MAX_LEN - len(segment_ids))
  test.append((torch.tensor(token_ids[:512]).long(),torch.tensor(attention_mask[:512]).long(),torch.tensor(segment_ids[:512]).long(),id))

##테스트셋 생성

In [ ]:
model = BERTClassifier(bertmodel,dr_rate=0.1).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/dacon_ExtSum/checkpoint_ext_bias_false.tar',map_location=torch.device('cpu'))['model_state_dict'])

<All keys matched successfully>

In [ ]:
test_set = torch.utils.data.DataLoader(test,batch_size=8)

In [ ]:
model.eval()
model.to(device)
sub = {}
for batch_id, (token_ids, attention_mask,token_type, id) in enumerate(tqdm_notebook(test_set)):
  token_ids = token_ids.to(device)
  attention_mask = attention_mask.to(device)
  token_type = token_type.to(device)
  output = model(token_ids, attention_mask, token_type)
  for i in range(len(id)) :
    idxs = []
    data = output[i]
    idx = data.argmax()
    idxs.append(idx.item())
    data[idx] = data.min()
    idx = data.argmax()
    idxs.append(idx.item())
    data[idx] = data.min()
    idx = data.argmax()
    idxs.append(idx.item())
    sub[id[i]] = idxs


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


## 예측

In [ ]:
for d in test_data :
  key = d['id']
  value = d['article_original']
  idx = sub[str(key)]
  text = '\n'.join([value[i] for i in idx])
  sub[str(key)] = text

In [ ]:
import pandas as pd
df2 = pd.DataFrame(data=sub.items(),columns=['id','summary'])

In [ ]:
df2.to_csv('/content/drive/MyDrive/dacon_ExtSum/extractive_20201203_70.csv',index=False)